In [42]:
# A Simple RAG Pipeline
import os
from dotenv import load_dotenv, dotenv_values
from utils import clean_text, print_chunks, rerank_chunks, reset_pinecone, embed_and_upsert, get_relevant_chunks

# Load All ENV Values
load_dotenv()

# 1. Load the txt file
with open('docs/the-judicial-system-of-pakistan.txt', 'r', encoding='utf-8') as file:
    # Read the content of the file
    text = file.read()


<figure>

![](figures/0)

<!-- FigureContent=\"FEDERAL JUDICIAL ACADEMY A Established 1988\" -->

</figure>


The Judicial System of Pakistan
===

<figure>

![](figures/1)

</figure>

 :selected:
Dr Faqir Hussain Director General Federal Judicial Academy Islamabad
<figure>

![](figures/2)

<!-- FigureContent=\"FEDERAL JUDICIAL ACAD Established 1988\" -->

</figure>


The Judicial System of Pakistan

Dr Faqir Hussain Director General Federal Judicial Academy Islamabad


||||
| - | - | - |
| 1 st edition | August | 2006 |
| 2nd edition | May | 2007 |
| 3rd edition | February | 2011 |
| 4th edition | May | 2015 |

<!-- PageHeader=\"The Judicial System of Pakistan\" -->


## Contents

| S. No | | Page |
| - | - | - |
| 1 | Introduction | 1 |
| 2 | Historical Retrospect | 1 |
| | 2.1 Hindu Period | 1 |
| | 2.2 Muslim Period | 2 |
| | 2.3 British Period | 4 |
| 3 | Post-Independence Evolution | 5 |
| 4 | Superior Judiciary | 6 |
| | 4.1 New Procedure for Appointment | 7 |
| | 4.2 Accountabil

In [40]:
print("----- Cleaned Text -----")
print(clean_text(text))

----- Cleaned Text -----
<figure>\n\n![](figures/0)\n\n<!-- FigureContent=\"FEDERAL JUDICIAL ACADEMY A Established 1988\" -->\n\n</figure>\n\n\nThe Judicial System of Pakistan\n===\n\n<figure>\n\n![](figures/1)\n\n</figure>\n\n :selected:\nDr Faqir Hussain Director General Federal Judicial Academy Islamabad\n<figure>\n\n![](figures/2)\n\n<!-- FigureContent=\"FEDERAL JUDICIAL ACAD Established 1988\" -->\n\n</figure>\n\n\nThe Judicial System of Pakistan\n\nDr Faqir Hussain Director General Federal Judicial Academy Islamabad\n\n\n||||\n| - | - | - |\n| 1 st edition | August | 2006 |\n| 2nd edition | May | 2007 |\n| 3rd edition | February | 2011 |\n| 4th edition | May | 2015 |\n\n<!-- PageHeader=\"The Judicial System of Pakistan\" -->\n\n\n## Contents\n\n| S. No | | Page |\n| - | - | - |\n| 1 | Introduction | 1 |\n| 2 | Historical Retrospect | 1 |\n| | 2.1 Hindu Period | 1 |\n| | 2.2 Muslim Period | 2 |\n| | 2.3 British Period | 4 |\n| 3 | Post-Independence Evolution | 5 |\n| 4 | Superior 

In [19]:
# 2. Create Chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=0)
# Create chunks, extract text
texts = list(map((lambda x: x.page_content),
             text_splitter.create_documents([clean_text(text)])))
print_chunks(texts)

Total Chunks:  98
--------
CHUNK 1
--------
Established 1988
The Judicial System
of Pakistan
Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad
Established 1988
The Judicial System
of Pakistan
Dr Faqir Hussain
Director General
Federal Judicial Academy
Islamabad
1st edition
August
nd edition
May
rd edition
February 2011
4th edition
May
2015
The Judicial System of Pakistan
Contents
S. No
--------
CHUNK 2
--------
Introduction
Historical Retrospect
2.1
Hindu Period
2.2
Muslim Period
2.3
British Period
Post-Independence Evolution
Superior Judiciary
4.1
New Procedure for Appointment
4.2
Accountability
4.3
Supreme Court
4.4
High Courts
4.5
Federal Shariat Court
Subordinate Courts
Special Courts and Administrative Tribunals
6.1
Service Tribunals
Procedural Law
Terms and Conditions of Service of Subordinate Judiciary
8.1
Workload
8.2
National Judicial Policy
Organization of Judicial Administration
9.1
Organogram of Courts
9.2
Strength of Judges
9.3
Superior Courts
9.4
Sub

In [20]:
# 3. Reset Pinecone and Upsert Chunks. Only Run Once
# reset_pinecone()
# embed_and_upsert(texts)

In [21]:
# https://chatgpt.com/share/271c8eea-0efc-4c6d-9e76-391a11eecc1b
queries = ["Describe the hierarchy and organizational structure of the judicial system in Pakistan",
           "Explain the reasons behind the discrepancy in the number of judges across different High Courts in Pakistan", 
           "What are the challenges faced by subordinate courts in Pakistan", 
           "Discuss the types of special courts and administrative tribunals in Pakistan and their jurisdictions", 
           "What are the procedural laws governing civil and criminal proceedings in Pakistan?", 
           "How does the appointment and independence of service tribunal members in Pakistan work?", 
           "Analyze the impact of the 2007 Provisional Constitution Order (PCO) on the strength of judges in the Supreme Court and High Courts of Pakistan", 
           "What is the role of the Federal Shariat Court in Pakistan's judicial system?", 
           "Summarize the annual performance of the judiciary in terms of case disposal and pendency as of 2013",
           "What is the significance of the Qanun-e-Shahadat Order 1984 in Pakistan’s legal proceedings?"]

query = queries[4]
relevant_chunks = get_relevant_chunks(query, 10)
relevant_chunks_text = [item["metadata"]["text"] for item in relevant_chunks]

print_chunks(relevant_chunks, True)

# The closest chunk to the queries[0] is on 7th number based on similarity, which is not good

Total Chunks:  10
--------
CHUNK 1
--------
Id: 59, Score: 0.84304148
and changing conditions of the society. Similarly, the Code of Criminal Procedure 1898 prescribes
the criminal procedure. The Qanun-e-Shahadat Order 1984 prescribes the competency of witnesses,
the examination of witnesses, form of evidence and the procedure for presenting the same. The
procedure prescribed in the law applies to judicial proceedings and investigations by a court of law
in civil or criminal cases. The special courts follow the procedure prescribed in the above-mentioned
codes, whereas some such courts follow special procedure, laid down in the respective statute.
8.
Terms and Conditions of Service of Subordinate Judiciary
The Subordinate Courts (civil and criminal) have been established and their jurisdiction defined by
law.61 They are supervised and controlled by the respective High Court.62 The administration of
justice, however, is a provincial subject and thus the Subordinate Courts are organised 

In [22]:
# Lets use a reranker to see how much it helps
rerank_results = rerank_chunks(query, relevant_chunks_text)
print_chunks(rerank_results, True)

Total Chunks:  10
--------
CHUNK 1
--------
Id: 0, Score: 0.00970458984375
and changing conditions of the society. Similarly, the Code of Criminal Procedure 1898 prescribes
the criminal procedure. The Qanun-e-Shahadat Order 1984 prescribes the competency of witnesses,
the examination of witnesses, form of evidence and the procedure for presenting the same. The
procedure prescribed in the law applies to judicial proceedings and investigations by a court of law
in civil or criminal cases. The special courts follow the procedure prescribed in the above-mentioned
codes, whereas some such courts follow special procedure, laid down in the respective statute.
8.
Terms and Conditions of Service of Subordinate Judiciary
The Subordinate Courts (civil and criminal) have been established and their jurisdiction defined by
law.61 They are supervised and controlled by the respective High Court.62 The administration of
justice, however, is a provincial subject and thus the Subordinate Courts are organ

In [23]:
# Only the Top Chunks should be given to llm, so we choose chunks with a score of 0.8
top_chunks = [item for item in rerank_results if item["score"] >= 0.6]
print_chunks(top_chunks, True)

Total Chunks:  0


<figure>

![](figures/0)

<!-- FigureContent="FEDERAL JUDICIAL ACADEMY A Established 1988" -->

</figure>


The Judicial System of Pakistan
===

<figure>

![](figures/1)

</figure>

 :selected:
Dr Faqir Hussain Director General Federal Judicial Academy Islamabad
<figure>

![](figures/2)

<!-- FigureContent="FEDERAL JUDICIAL ACAD Established 1988" -->

</figure>


The Judicial System of Pakistan

Dr Faqir Hussain Director General Federal Judicial Academy Islamabad


||||
| - | - | - |
| 1 st edition | August | 2006 |
| 2nd edition | May | 2007 |
| 3rd edition | February | 2011 |
| 4th edition | May | 2015 |

<!-- PageHeader="The Judicial System of Pakistan" -->


## Contents

| S. No | | Page |
| - | - | - |
| 1 | Introduction | 1 |
| 2 | Historical Retrospect | 1 |
| | 2.1 Hindu Period | 1 |
| | 2.2 Muslim Period | 2 |
| | 2.3 British Period | 4 |
| 3 | Post-Independence Evolution | 5 |
| 4 | Superior Judiciary | 6 |
| | 4.1 New Procedure for Appointment | 7 |
| | 4.2 Accountability | 